In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

### Connections Scraper

In [2]:
def browser_wait(driver, element, locator=By.XPATH, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((locator, element))
    )

def browser_wait_multi(driver, element, locator=By.XPATH, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_all_elements_located((locator, element))
    )

def scroll(driver, start=0, end='document.body.scrollHeight'):
    driver.execute_script(f'window.scrollTo({start}, {end});')    

In [3]:
# env
USERNAME = 'fahri.lafa@gmail.com'
PASSWORD = '1qw23er4'

In [ ]:
# specify chrome options
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)

# open the site
print('Opening site...')
browser.get('https://linkedin.com')

# move to login page
try:    
    login_btn = browser_wait(browser, '/html/body/nav/div/a[2]')
    login_btn.click()
except:
    print('Cannot find login button!')

# fill credentials
print('Attempting to login...')
try:
    email_field = browser_wait(browser, '#username', locator=By.CSS_SELECTOR)
    pwd_field = browser_wait(browser, '#password', locator=By.CSS_SELECTOR)
    # insert email password
    email_field.send_keys(USERNAME)
    pwd_field.send_keys(PASSWORD)
    # submit
    time.sleep(1.5)
    pwd_field.send_keys(Keys.RETURN)
    print('Submit login...')
except:
    print('Cannot find email or passowrd field!')

# go to My Network page
print('Move to `My Network` page')
browser.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')

# scroll to bottom
last_height = browser.execute_script('return document.body.scrollHeight')
wait_scroll = 5
while True:
    print('Scrolling...')
    # scroll until the last record
    scroll(browser)
    print('Wait 5s...')
    time.sleep(5)
    new_height = browser.execute_script('return document.body.scrollHeight')
    # if scroll reach the end
    if new_height == last_height:
        try:
            # look for load_more button
            print('Looking for load more button...')
            # load_more = browser_wait(browser, '/html/body/div[7]/div[3]/div/div/div/div/div[2]/div/div/main/div/section/div[2]/div[2]/div/button')
            load_more = browser_wait(browser, 'scaffold-finite-scroll__load-button', locator=By.CLASS_NAME)
            load_more.click()
            print('Wait 5s...')
            time.sleep(5)
            # scroll again
            print('Scrolling...')
            scroll(browser)
        except:
            # no more data
            print('Load more button not found! It seems we are reached the end')
            break
    # update last height
    print('Update last height')
    last_height = new_height
            
# get raw html
print('Get output...')
my_networks = BeautifulSoup(browser.page_source.encode('utf-8'), 'html')
my_networks.prettify()

# write html to file
print('Write output...')
with open('my_networks.html', 'w', encoding='utf-8') as w:
    w.write(str(my_networks))

# # anaylzing html tags
# print('Analyzing html tags...')

# quit browser
print('Done. Quitting...')
time.sleep(5)

browser.close()

In [17]:
containers = my_networks.findAll('li', {'class': 'mn-connection-card artdeco-list'})
len(containers)

839

In [18]:
# parsing profiles list
profiles = []

for c in containers:
    # data model
    m_profile = {
        'name': None,
        'occupation': None,
        'profile_link': 'https://linkedin.com{}',
        'connected_at': None
    }
    m_profile['name'] = c.find('span', {'class': 'mn-connection-card__name'}).text.strip()
    m_profile['occupation'] = c.find('span', {'class': 'mn-connection-card__occupation'}).text.strip()
    m_profile['profile_link'] = m_profile['profile_link'].format(c.find('a', {'class': 'mn-connection-card__link'})['href'])
    m_profile['connected_at'] = c.find('time').text.strip().split('\n')[-1].strip()
    profiles.append(m_profile)    

df_profile = pd.DataFrame(profiles)
df_profile.head()

,name,occupation,profile_link,connected_at
0,"Sie, Evan Setiawan",Class Coordinator - Sekolah Data PACMANN,https://linkedin.com/in/sie-evan-setiawan/,1 day ago
1,Erika Dwi Puspitasari,Student at Airlangga University,https://linkedin.com/in/erika-dwi-puspitasari-...,2 days ago
2,Ekaterina Shlenova,IT Recruiter in Yandex,https://linkedin.com/in/ekaterina-shlenova-714...,3 days ago
3,eef hikmat,Student at sekolah tinggi ilmu ekonomi 66,https://linkedin.com/in/eef-hikmat-13332a1a6/,1 week ago
4,Amna Maharani,IT Student at Fachhochschule Kiel,https://linkedin.com/in/amaharani/,1 week ago


### Profile Scraper

In [10]:
df_profile.profile_link

0             https://linkedin.com/in/sie-evan-setiawan/
1      https://linkedin.com/in/erika-dwi-puspitasari-...
2      https://linkedin.com/in/ekaterina-shlenova-714...
3          https://linkedin.com/in/eef-hikmat-13332a1a6/
4                     https://linkedin.com/in/amaharani/
                             ...                        
834                  https://linkedin.com/in/nabilah-iz/
835    https://linkedin.com/in/zikra-halim-arrassy-a4...
836                       https://linkedin.com/in/magaz/
837    https://linkedin.com/in/soegianto-soelistiono-...
838    https://linkedin.com/in/cendra-devayana-putra-...
Name: profile_link, Length: 839, dtype: object

In [5]:
from actions import Actions
from credentials import Credentials
import constant

c = Credentials()
c.get_uid()

1

In [21]:
# specify chrome options
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
# options.add_argument('--headless')
browser = webdriver.Chrome(options=options)

actions = Actions(browser, c)
actions.do_login()

In [22]:
browser.get('https://linkedin.com/in/hanifizzudinr/')

## Getting Courses data

In [24]:
anchors = browser_wait_multi(browser, 'a.optional-action-target-wrapper.artdeco-button.artdeco-button--tertiary', By.CSS_SELECTOR, 30)
print(anchors)

[<selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="c9ca0d6e-8228-4732-82f5-8a945c7df062")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="b209d579-0290-4c80-8f3f-8a2880f2be1b")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="e9ce0923-41df-4996-9a6c-923a725f97f1")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="b36001d7-d539-42a7-9cdc-6ec2e2be2aba")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="557596c9-2df5-4396-ac43-3cb55a3d28de")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="8bd88188-7290-4eb2-a3e5-b4c164a9f269")>, <selenium.webdriver.remote.webelement.WebElement (session="effa5cf42a171f75f29db1cff97343a2", element="23073d50-a9fc-44a0-9cc5-6b035214

In [27]:
for a in anchors:
    btn = a.find_element(By.CSS_SELECTOR, 'span.pvs-navigation__text')
    print(btn.text)
    print(a.get_attribute('href'))
    # if 'licenses' in a.text:
    #     a.click()
    #     break

Show all activity
https://www.linkedin.com/in/hanifizzudinr/recent-activity/
Show all 7 experiences
https://www.linkedin.com/in/hanifizzudinr/details/experience?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABbAq48B4hyhD22ZfE3c0edd4IrVGQIYzKM
Show all 18 licenses & certifications
https://www.linkedin.com/in/hanifizzudinr/details/certifications?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABbAq48B4hyhD22ZfE3c0edd4IrVGQIYzKM
Show all 5 volunteer experiences
https://www.linkedin.com/in/hanifizzudinr/details/volunteering-experiences?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABbAq48B4hyhD22ZfE3c0edd4IrVGQIYzKM
Show all 40 skills
https://www.linkedin.com/in/hanifizzudinr/details/skills?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABbAq48B4hyhD22ZfE3c0edd4IrVGQIYzKM
Show all 7 courses
https://www.linkedin.com/in/hanifizzudinr/details/courses?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABbAq48B4hyhD22ZfE3c0edd4IrVGQIYzKM
Show all 15 projects
https://www.linkedin.com/in/hanifizzudinr/details/projects?profileUrn=urn%

In [11]:
keyword = 'experiences'
for btn in anchors:
    re_format = f'\d+ {keyword}$'
    if re.compile(re_format).search(btn.text):
        btn.click()        
        found_btn = True
        break

In [51]:
raw = BeautifulSoup(browser.page_source.encode('utf-8'), 'lxml')

In [42]:
def __experience_parser(raw_html, w_button=True):
    # get experiences
    if not w_button:                
        experiences = raw_html.select('#experience + div + div > ul > li.artdeco-list__item > div > div:nth-child(2)')
    else:
        experiences = raw_html.select('ul > li.artdeco-list__item > div > div:nth-child(2)')
        #  > div > div.display-flex
    
    exp_names = []
    for e, exp in enumerate(experiences):        
        nest = exp.select('div:nth-child(2) > ul > li > div.pvs-entity')
        if len(nest) > 0:
            # nested experience
            header = exp.select_one('div > div.display-flex > a')
            company = header.select_one('div.display-flex > span > span').text
            try:
                location = header.select_one('div + span + span > span').text
            except:
                location = None
            # scrap data
            for n in nest:
                position = n.select_one('div > span > span').text
                duration = n.select_one('div + span > span').text

                item = {
                    'company': company,
                    'position': position,
                    'duration': duration,
                    'location': location
                }
                exp_names.append(item)
        else:
            # non-nested experience
            exp = exp.select_one('div > div.display-flex')
            position = exp.select_one('div > span > span').text
            company = exp.select_one('div + span > span').text
            duration = exp.select_one('div + span + span > span').text
            try:
                location = exp.select_one('div + span + span + span > span').text
            except:
                location = None

            item = {
                'company': company,
                'position': position,
                'duration': duration,
                'location': location
            }

            exp_names.append(item)

    return exp_names

__experience_parser(raw, w_button=False)

[{'company': 'Jakarta Artificial Intelligence Research · Full-time',
  'position': 'Data Science Fellow',
  'duration': 'Oct 2021 - Present · 8 mos',
  'location': 'Jakarta, Indonesia'},
 {'company': 'Mitra Informatika · Full-time',
  'position': 'Data Engineer',
  'duration': 'Dec 2020 - Jun 2021 · 7 mos',
  'location': 'Surabaya, East Java, Indonesia'},
 {'company': 'Developer Student Clubs',
  'position': 'Leader, DSC UNAIR',
  'duration': 'Jul 2019 - Jul 2020 · 1 yr 1 mo',
  'location': 'Surabaya, East Java, Indonesia'},
 {'company': 'Developer Student Clubs',
  'position': 'Android Developer Mentor, DSC PENS',
  'duration': 'Apr 2018 - Jun 2019 · 1 yr 3 mos',
  'location': 'Surabaya, East Java, Indonesia'}]

In [53]:
def __course_parser(raw_html, w_button=True):            
    # get courses
    if not w_button:
        courses = raw_html.select('#licenses_and_certifications + div + div > ul > li.artdeco-list__item > div > div:nth-child(2) > div > a')
    else:                
        courses = raw_html.select('ul > li.artdeco-list__item > div > div:nth-child(2) > div > a')

    course_names = []
    for c in courses:
        # course name
        name = c.select_one('div > span > span').text
        # issuing organization
        organization = c.select_one('div + span > span').text

        item = {
            'course': name,
            'organization': organization
        } 

        course_names.append(item)
    
    return course_names

__course_parser(raw, w_button=True)

[{'course': 'Introduction to Tableau', 'organization': 'DataCamp'},
 {'course': 'Analyzing Police Activity with pandas',
  'organization': 'DataCamp'},
 {'course': 'Cleaning Data in Python', 'organization': 'DataCamp'},
 {'course': 'Exploratory Data Analysis in Python', 'organization': 'DataCamp'},
 {'course': 'Introduction to SQL', 'organization': 'DataCamp'},
 {'course': 'Joining Data in SQL', 'organization': 'DataCamp'},
 {'course': 'Streamlined Data Ingestion with pandas',
  'organization': 'DataCamp'},
 {'course': 'Data Analysis Using Python', 'organization': 'IBM'},
 {'course': 'Data Science for Business - Level 1', 'organization': 'IBM'},
 {'course': 'Data Visualization Using Python', 'organization': 'IBM'},
 {'course': 'Deep Learning Essentials', 'organization': 'IBM'},
 {'course': 'Enterprise Design Thinking Practitioner', 'organization': 'IBM'},
 {'course': 'Intermediate Importing Data in Python',
  'organization': 'DataCamp'},
 {'course': 'Introduction to Data Visualization 

In [33]:
experiences = raw.select('ul > li.artdeco-list__item > div > div:nth-child(2)')
experiences[0].select_one('div > div.display-flex')

<div class="display-flex align-items-center">
<span class="mr1 t-bold">
<span aria-hidden="true"><!-- -->Tech Skills Day 2022<!-- --></span><span class="visually-hidden"><!-- -->Tech Skills Day 2022<!-- --></span>
</span>
<!-- --><!-- --><!-- --> </div>

In [61]:
courses = raw.select('ul > li.artdeco-list__item > div > div:nth-child(2) > div > a')
# courses[0].select_one('div > span > span:nth-child(1)').text
courses[0].select_one('div > span > span').text

'Machine Learning with Python'

In [106]:
# scrap courses in the profile page
courses = raw.select('#licenses_and_certifications + div + div > ul > li.artdeco-list__item > div > div:nth-child(2) > div > a')
courses[0].select_one('div > span > span').text

'Machine Learning with Python'

In [112]:
browser.get_cookie('li_at')['value']

'AQEDARyUVaEDoLC2AAABgOea8hgAAAGBC6d2GFYAuCMqwDxnIg3N6QAYWzN9K9bzR0kd5Ti8HhIZUx9KpfeWvFjx4OOL9je3bfzVaahYYYUC7s0UD3C-I9TfFp-drdruO922tN7AGx-QCslAARRsluxq'

In [ ]:
# scrap educations in the profile page
edus = raw.select('#education + div + div > ul > li.artdeco-list__item > div > div:nth-child(2) > div > a')
edus

## Getting Experiences data

In [43]:
browser_wait(browser, '#licenses_and_certifications', By.CSS_SELECTOR, 15)

TimeoutException: Message: 


### Regular format experience selector in profile page

In [38]:
# scrap experience in the profile page
exps = raw.select('#experience + div + div > ul > li.artdeco-list__item > div > div:nth-child(2) > div > div.display-flex')
print(exps[0].select_one('div > span > span').text) # position
print(exps[0].select_one('div + span > span').text) # company
print(exps[0].select_one('div + span + span > span').text) # duration
print(exps[0].select_one('div + span + span + span > span').text) # location

Data Science Fellow
Jakarta Artificial Intelligence Research · Full-time
Oct 2021 - Present · 8 mos
Jakarta, Indonesia


### Nested experience selector in profile page

In [52]:
# in the profile page
exps = raw.select('#experience + div + div > ul > li.artdeco-list__item > div > div:nth-child(2)')
nest = exps[2].select('div:nth-child(2) > ul > li > div.pvs-entity')
print(nest[0].select_one('div > span > span').text)     # position
print(nest[0].select_one('div + span > span').text)     # duration

Leader, DSC UNAIR
Jul 2019 - Jul 2020 · 1 yr 1 mo


In [79]:
header = exps[2].select_one('div > div.display-flex > a')
header.select_one('div.display-flex > span > span').text
header.select_one('div + span + span > span').text

'Surabaya, East Java, Indonesia'

### Regular format experience selector in Detail Experience

In [8]:
# scrap experience in detail experience page
anchors = browser_wait_multi(browser, 'a.optional-action-target-wrapper.artdeco-button.artdeco-button--tertiary > span.pvs-navigation__text', By.CSS_SELECTOR, 30)
for a in anchors:
    if 'experience' in a.text:
        a.click()
        break

In [13]:
raw = BeautifulSoup(browser.page_source.encode('utf-8'), 'lxml')
exps = raw.select('ul > li.artdeco-list__item > div > div:nth-child(2) > div > div.display-flex')
print(exps[1].select_one('div > span > span').text) # position
print(exps[1].select_one('div + span > span').text) # company
print(exps[1].select_one('div + span + span > span').text) # duration
print(exps[1].select_one('div + span + span + span > span').text) # location

Analista de pruebas de software
CENET SA · Full-time
Apr 2016 - Apr 2018 · 2 yrs 1 mo
Bogotá D.C., Colombia


## Getting connections of connection

In [49]:
link = browser_wait(browser, 'li.text-body-small > a', By.CSS_SELECTOR, 30)
print(link)
link.click()

<selenium.webdriver.remote.webelement.WebElement (session="4ccc3bea51fead360bab2cf6320b1301", element="c2fe6bdf-cbf0-493f-9e93-6906e9058ae6")>


In [50]:
try:
    # trigger filter dropdown
    filter_btn = browser_wait(
        browser, 
        'div.search-reusables__filter-trigger-and-dropdown:nth-child(1)', 
        By.CSS_SELECTOR, 
        timeout=30
    )
    filter_btn.click()

    # disable 1st circle connection
    first_circle = browser_wait(
        browser, 
        'li.search-reusables__collection-values-item:nth-child(1) > label', 
        By.CSS_SELECTOR, 
        timeout=30
    )
    first_circle.click()

    # enable 3rd circle connection
    third_circle = browser_wait(
        browser, 
        'li.search-reusables__collection-values-item:nth-child(3) > label', 
        By.CSS_SELECTOR, 
        timeout=30
    )
    third_circle.click()

    # submit option
    submit = browser_wait(browser, '/html/body/div[7]/div[3]/div[2]/section/div/nav/div/ul/li[3]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]')
    submit.click()
except:
    print('There is a problem when we trying to filter connection to 2nd and 3rd circle.')

In [53]:
last_height = browser.execute_script('return document.body.scrollHeight')
while True:    
    # scroll until the last record
    scroll(browser, end=1000)    
    time.sleep(2)
    new_height = browser.execute_script('return document.body.scrollHeight')
    # if scroll reach the end
    if new_height == last_height:        
        break
    
    last_height = new_height

In [54]:
btn_class = 'button.artdeco-pagination__button--next'
# click next button
next_btn = browser_wait(
    browser,
    btn_class,
    locator=By.CSS_SELECTOR,
    timeout=50
)
# check if button is clickable
is_disabled = browser.execute_script(
    f'return document.querySelector("{btn_class}").classList.contains("artdeco-button--disabled")'
)
print(is_disabled)
# next_btn = is_element_clickable(
#     browser, 
#     'button.artdeco-pagination__button--next',
#     locator=By.CSS_SELECTOR,
#     timeout=50
# )

if not is_disabled:
    # button still clickable
    next_btn.click()
    print('NEXT_PAGE')
else:
    print('END_PAGE')

False
NEXT_PAGE


In [119]:
browser.get('https://linkedin.com/in/sie-evan-setiawan/')
# link = browser_wait(browser, 'link-without-visited-state:nth-child(2)', locator=By.CLASS_NAME, timeout=120)
# WebDriverWait(browser, 10)
# link = browser.find_element(By.XPATH, '/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/ul/li/a')
# link = WebDriverWait(browser, 30).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'li.text-body-small:nth-child(2)'))
#     )
link = browser_wait(browser, 'li.text-body-small:nth-child(2)', By.CSS_SELECTOR, 30)
print(link)

<selenium.webdriver.remote.webelement.WebElement (session="82bb0f6927ca24d3dc34122684cb4d0a", element="faf6ec61-aeb2-4876-8e4c-f0b0ad1af7e1")>


In [72]:
link.click()

In [75]:
filter_btn = browser_wait(browser, 'div.search-reusables__filter-trigger-and-dropdown:nth-child(1)', By.CSS_SELECTOR, timeout=30)
filter_btn.click()

first_circle = browser_wait(browser, 'li.search-reusables__collection-values-item:nth-child(1) > label', By.CSS_SELECTOR, timeout=30)
first_circle.click()
third_circle = browser_wait(browser, 'li.search-reusables__collection-values-item:nth-child(3) > label', By.CSS_SELECTOR, timeout=30)
third_circle.click()

In [67]:
submit = browser_wait(browser, '/html/body/div[7]/div[3]/div[2]/section/div/nav/div/ul/li[3]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]')
submit.click()

In [68]:
next = browser_wait(browser, '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[2]')
next

<selenium.webdriver.remote.webelement.WebElement (session="3bec0805c7b54e7b05113556ffb54bfa", element="8969d98c-084f-44f7-8ecf-4962b03821fa")>

In [122]:
WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.artdeco-pagination__button--next')))
                                        #       '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[2]'
                                        #       '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div/div[2]/div/button[2]'
        # EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[1]')))

<selenium.webdriver.remote.webelement.WebElement (session="82bb0f6927ca24d3dc34122684cb4d0a", element="1b0c618d-41bb-4be9-aa41-0ddf2753ea8d")>

In [ ]:
# scrap item
connections = BeautifulSoup(browser.page_source.encode('utf-8'), 'html')
connections.prettify()

In [80]:
containers = connections.findAll('div', {'class': 'entity-result__item'})
len(containers)

10

In [109]:
for c in containers:
    heading = c.find('span', {'class': 'entity-result__title-text t-16'})    
    # data properties
    name = heading.find('span', {'dir': 'ltr'}).span.text
    circle_level = heading.find('span', {'class': 'image-text-lockup__text entity-result__badge-text'}).span.text.split()[-1][0]
    profile_link = heading.a['href']
    occupation = c.find('div', {'class': 'entity-result__primary-subtitle t-14 t-black t-normal'}).text.strip()
    
    break

Ladinda Tasya


In [4]:
import threading
from queue import Queue
import time

In [5]:
qlist = []

for i in range(5):
    # specify chrome options
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-extensions')
    # options.add_argument('--headless')
    browser = webdriver.Chrome(options=options)
    
    actions = Actions(browser, c)
    actions.do_login()

    qlist.append(browser)

len(qlist)

WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...


5

In [7]:
from objectpool import ObjectPool
pool = ObjectPool.get_instance()
pool.set_resource(qlist)
pool.get_total_resource()

5

In [15]:
links = [
    'https://linkedin.com/in/sie-evan-setiawan/',
    'https://linkedin.com/in/erika-dwi-puspitasari-965b7318b/',
    'https://linkedin.com/in/ekaterina-shlenova-714504189/',
    'https://linkedin.com/in/eef-hikmat-13332a1a6/',
    'https://linkedin.com/in/amaharani/',
    'https://linkedin.com/in/nemat-allah-aloush-60887a1a9/',
    'https://linkedin.com/in/rayhanozzy/',
    'https://linkedin.com/in/reynaldipriandika/',
    'https://linkedin.com/in/faisal-ichsan/',
    'https://linkedin.com/in/sisca-fahrudha-retno-agustina-119396183/',
    'https://linkedin.com/in/muhammad-abil-nurjani/',
    'https://linkedin.com/in/mishrag37/',
    'https://linkedin.com/in/rahmatri-mardiko-56720690/',
    'https://linkedin.com/in/bintang-muhammad-326827140/',
    'https://linkedin.com/in/hanifizzudinr/',
    'https://linkedin.com/in/frdaamalia/',
    'https://linkedin.com/in/andrey-kiselevich-368165232/',
    'https://linkedin.com/in/masetarahma/',
    'https://linkedin.com/in/ika-sulistiorini-a3032a152/',
    'https://linkedin.com/in/cerelia-devina-tantia-vinci-ninggalih-1949571b6/',
    'https://linkedin.com/in/ilhamnurakbar/',
    'https://linkedin.com/in/wahyu-nur-abidin-b20b26222/',
    'https://linkedin.com/in/houssam-hammoudi/',
    'https://linkedin.com/in/vladislav-vasiljev-a3a22021a/',
    'https://linkedin.com/in/ayu-anggraini30/'
]

In [16]:
lock_db = threading.Lock()

def dumper_thread(profile_link, op, thread_name):
    while True:
        print(f'{thread_name} borrowing object...')
        driver = op.borrow_resource()
        print(f'{thread_name} opening link...')
        driver.get(profile_link)
        print(f'{thread_name} dumping html...')
        # TODO: dump html
        with lock_db:
            print(f'{thread_name} inserting to db...')
            time.sleep(10)
        print(f'{thread_name} returning resource...')
        op.return_resource(driver)
        print(f'{thread_name} done.')
        break

threads = []
for i, item in enumerate(links):
    thread_name = f'Thread-{i}'
    t = threading.Thread(target=dumper_thread, args=(item, pool, thread_name))
    t.name = thread_name
    threads.append(t)

In [19]:
max_thread_exec = 5
curr_thread = 0

for _ in range(max_thread_exec):
    threads[curr_thread].start()
    curr_thread += 1

Thread-24 borrowing object...
Thread-24 opening link...


RuntimeError: threads can only be started once

Thread-24 dumping html...
Thread-24 inserting to db...
Thread-24 returning resource...
Thread-24 done.


In [18]:
curr_thread

5

In [39]:
insert_count = 0

def dumper_thread(queue):
    while True:        
        try:
            i, item = queue.get()
            thread_name = f'Thread-{i}'
            print(f'{thread_name} borrowing resource...')
            driver = pool.borrow_resource()
            print(f'{thread_name} opening link...')
            driver.get(item)
            print(f'{thread_name} dumping page...')                
            time.sleep(3)

            with lock_db:
                print(f'{thread_name} inserting to db...')
                time.sleep(10)

            print(f'{thread_name} done.')
            queue.task_done()
            
        except:
            raise SystemError('No free resource currently.')
    

for _ in range(5):    
    t = threading.Thread(target=dumper_thread, args=(qlinks))
    t.start()

Exception in thread Thread-74:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-75:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-76:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-77:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
        self.run()
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
self.run()
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
      File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
Exception in thread Thread-78:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", li

In [30]:
lock = threading.Lock()
count = 0

def thread_test(i):
    print(f'Thread-{i} attempting to update count...')
    with lock:
        global count
        count += 1
        print(f'Count was updated by thread-{i}!')
        time.sleep(10)

def threader():
    while True:
        i = q.get()
        print(f'Thread-{i} starting...')
        thread_test(i)
        time.sleep(3)
        print(f'Thread-{i} done.')
        q.task_done()

q = Queue()
for x in range(5):
    thread = threading.Thread(target=threader)

    # thread.daemon = True
    thread.start()

for job in range(10):
    q.put(job)

Thread-0 starting...
Thread-0 attempting to update count...
Count was updated by thread-0!
Thread-1 starting...
Thread-1 attempting to update count...
Thread-2 starting...
Thread-2 attempting to update count...
Thread-3 starting...
Thread-3 attempting to update count...
Thread-4 starting...
Thread-4 attempting to update count...
Count was updated by thread-1!
Thread-0 done.
Thread-5 starting...
Thread-5 attempting to update count...
Count was updated by thread-2!
Thread-1 done.
Thread-6 starting...
Thread-6 attempting to update count...
Count was updated by thread-3!
Thread-2 done.
Thread-7 starting...
Thread-7 attempting to update count...
Count was updated by thread-4!
Thread-3 done.
Thread-8 starting...
Thread-8 attempting to update count...
Count was updated by thread-5!
Thread-4 done.
Thread-9 starting...
Thread-9 attempting to update count...
Count was updated by thread-6!
Thread-5 done.
Count was updated by thread-7!
Thread-6 done.
Count was updated by thread-8!
Thread-7 done.
C

In [27]:
print(count)

10


In [3]:
import pandas as pd
import numpy as np
import json

exp = [{'company': 'Jakarta Artificial Intelligence Research · Full-time',
  'position': 'Data Science Fellow',
  'duration': 'Oct 2021 - Present · 8 mos',
  'location': 'Jakarta, Indonesia'},
 {'company': 'Mitra Informatika · Full-time',
  'position': 'Data Engineer',
  'duration': 'Dec 2020 - Jun 2021 · 7 mos',
  'location': 'Surabaya, East Java, Indonesia'},
 {'company': 'Developer Student Clubs',
  'position': 'Leader, DSC UNAIR',
  'duration': 'Jul 2019 - Jul 2020 · 1 yr 1 mo',
  'location': 'Surabaya, East Java, Indonesia'},
 {'company': 'Developer Student Clubs',
  'position': 'Android Developer Mentor, DSC PENS',
  'duration': 'Apr 2018 - Jun 2019 · 1 yr 3 mos',
  'location': 'Surabaya, East Java, Indonesia'}]

courses = [{'course': 'Introduction to Tableau', 'organization': 'DataCamp'},
 {'course': 'Analyzing Police Activity with pandas',
  'organization': 'DataCamp'},
 {'course': 'Cleaning Data in Python', 'organization': 'DataCamp'},
 {'course': 'Exploratory Data Analysis in Python', 'organization': 'DataCamp'},
 {'course': 'Introduction to SQL', 'organization': 'DataCamp'},
 {'course': 'Joining Data in SQL', 'organization': 'DataCamp'},
 {'course': 'Streamlined Data Ingestion with pandas',
  'organization': 'DataCamp'},
 {'course': 'Data Analysis Using Python', 'organization': 'IBM'},
 {'course': 'Data Science for Business - Level 1', 'organization': 'IBM'},
 {'course': 'Data Visualization Using Python', 'organization': 'IBM'},
 {'course': 'Deep Learning Essentials', 'organization': 'IBM'},
 {'course': 'Enterprise Design Thinking Practitioner', 'organization': 'IBM'},
 {'course': 'Intermediate Importing Data in Python',
  'organization': 'DataCamp'},
 {'course': 'Introduction to Data Visualization with Matplotlib',
  'organization': 'DataCamp'},
 {'course': 'Introduction to Data Visualization with Seaborn',
  'organization': 'DataCamp'},
 {'course': 'Introduction to Importing Data in Python',
  'organization': 'DataCamp'},
 {'course': 'Joining Data with pandas', 'organization': 'DataCamp'},
 {'course': 'Machine Learning with Python - Level 1', 'organization': 'IBM'},
 {'course': 'Python for Data Science', 'organization': 'IBM'},
 {'course': 'Big Data Foundations - Level 1', 'organization': 'IBM'},
 {'course': 'Cloud Core', 'organization': 'IBM'},
 {'course': 'Data Manipulation with pandas', 'organization': 'DataCamp'},
 {'course': 'Data Science Foundations - Level 1', 'organization': 'IBM'},
 {'course': 'Data Science Foundations - Level 2 (V2)', 'organization': 'IBM'},
 {'course': 'Data Science Methodologies', 'organization': 'IBM'},
 {'course': 'Introduction to Data Science in Python',
  'organization': 'DataCamp'},
 {'course': 'Watson Studio Essentials', 'organization': 'IBM'},
 {'course': 'Intermediate Python', 'organization': 'DataCamp'},
 {'course': 'Belajar Machine Learning untuk Pemula',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Memulai Pemrograman Dengan Python',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Belajar Prinsip Pemrograman SOLID',
  'organization': 'Dicoding Academy'},
 {'course': 'Memulai Pemrograman Dengan Kotlin',
  'organization': 'Dicoding Academy'},
 {'course': 'Belajar Membuat Aplikasi Android untuk Pemula',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Menjadi Azure Cloud Developer',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Belajar Membangun Prototype Chatbot Dengan LINE Bot Designer',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Kotlin Android Developer Expert',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Belajar Membuat Aplikasi Kognitif',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Belajar Fundamental Aplikasi Android',
  'organization': 'Dicoding Indonesia'},
 {'course': 'Belajar Membangun LINE Chatbot',
  'organization': 'Dicoding Indonesia'}]

data = {
  'data': [{
    'name': 'Affandy Fahrizain',
    'link': 'https://www.linkedin.com/in/fahrizainn/',
    'experiences': exp,
    # 'courses': courses
  }]
}

# json.dumps(data)
# pd.DataFrame(data)
# pd.read_json(json.dumps(data))
jj = json.dumps(data)
jl = json.loads(jj)
jl['data'].append({
    'name': 'Affandy',
    'link': 'https://www.linkedin.com/in/fahrizainn/',
    'experiences': exp,
    # 'courses': courses
  })
ju = json.dumps(jl)

jj

'{"data": [{"name": "Affandy Fahrizain", "link": "https://www.linkedin.com/in/fahrizainn/", "experiences": [{"company": "Jakarta Artificial Intelligence Research \\u00b7 Full-time", "position": "Data Science Fellow", "duration": "Oct 2021 - Present \\u00b7 8 mos", "location": "Jakarta, Indonesia"}, {"company": "Mitra Informatika \\u00b7 Full-time", "position": "Data Engineer", "duration": "Dec 2020 - Jun 2021 \\u00b7 7 mos", "location": "Surabaya, East Java, Indonesia"}, {"company": "Developer Student Clubs", "position": "Leader, DSC UNAIR", "duration": "Jul 2019 - Jul 2020 \\u00b7 1 yr 1 mo", "location": "Surabaya, East Java, Indonesia"}, {"company": "Developer Student Clubs", "position": "Android Developer Mentor, DSC PENS", "duration": "Apr 2018 - Jun 2019 \\u00b7 1 yr 3 mos", "location": "Surabaya, East Java, Indonesia"}]}]}'

In [28]:
with open('data.json', 'a+', encoding='utf-8') as f:
    json.dump(data, f)

In [7]:
import json
with open('data.json', 'r+', encoding='utf-8') as f:
    data = json.load(f)
    print(data)
    # data['data'].append({
    #     'name': 'Affandy',
    #     'link': 'https://www.linkedin.com/in/fahrizainn/',
    #     'experiences': exp,
    #     # 'courses': courses
    # })
    # f.seek(0)
    # json.dump(data, f, indent=4)

[{'name': 'eef hikmat', 'occupation': 'Student at sekolah tinggi ilmu ekonomi 66', 'profile_link': 'https://linkedin.com/in/eef-hikmat-13332a1a6/', 'circle_level': '1', 'connected_w_user_id': 1, 'connected_w_connection_id': None, 'experiences': None, 'courses': None}, {'name': 'Erika Dwi Puspitasari', 'occupation': 'Student at Airlangga University', 'profile_link': 'https://linkedin.com/in/erika-dwi-puspitasari-965b7318b/', 'circle_level': '1', 'connected_w_user_id': 1, 'connected_w_connection_id': None, 'experiences': None, 'courses': None}, {'name': 'Ekaterina Shlenova', 'occupation': 'HR Manager in Yandex', 'profile_link': 'https://linkedin.com/in/ekaterina-shlenova-714504189/', 'circle_level': '1', 'connected_w_user_id': 1, 'connected_w_connection_id': None, 'experiences': [{'company': 'Yandex', 'position': 'Human Resources Project Manager', 'duration': 'Apr 2022 - Present · 2 mos', 'location': None}, {'company': 'Yandex', 'position': 'IT Recruiter', 'duration': 'Mar 2021 - Present

In [30]:
import os.path

os.path.exists('./data.json')

True

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\fahrizain\\anaconda3\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()
spark

In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
tup_data = [
    (
        'Affandy Fahrizain', 
        'https://www.linkedin.com/in/fahrizainn/',
        (
            'Jakarta Artificial Intelligence Research · Full-time',
            'Data Science Fellow',
            'Oct 2021 - Present · 8 mos',
            'Jakarta, Indonesia'
        )
    )
]

In [6]:
from pyspark.sql.types import StructType, StructField, StringType
scheme = StructType([
    StructField('name', StringType(), True),
    StructField('link', StringType(), True),
    StructField('experiences', StructType([
        StructField('company', StringType(), True),
        StructField('position', StringType(), True),
        StructField('duration', StringType(), True),
        StructField('location', StringType(), True)
    ]))
])

spark_df = spark.createDataFrame(data=tup_data, schema=scheme)
spark_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- link: string (nullable = true)
 |-- experiences: struct (nullable = true)
 |    |-- company: string (nullable = true)
 |    |-- position: string (nullable = true)
 |    |-- duration: string (nullable = true)
 |    |-- location: string (nullable = true)



In [12]:
spark_df.show()

+-----------------+--------------------+--------------------+
|             name|                link|         experiences|
+-----------------+--------------------+--------------------+
|Affandy Fahrizain|https://www.linke...|{Jakarta Artifici...|
+-----------------+--------------------+--------------------+



In [15]:
spark_df.select('experiences').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+
|experiences                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------------+
|{Jakarta Artificial Intelligence Research · Full-time, Data Science Fellow, Oct 2021 - Present · 8 mos, Jakarta, Indonesia}|
+---------------------------------------------------------------------------------------------------------------------------+



# BUG
```
[hanif] Message: element click intercepted: Element <span class="pvs-navigation__text">...</span> is not clickable at point (337, 10). Other element would receive the click: <section class="artdeco-global-alert__body" data-test-global-alert-body="0">...</section>                                                                              (Session info: chrome=101.0.4951.67)                                                                           Stacktrace:                                                                                                      Backtrace:                                                                                                               Ordinal0 [0x003AB8F3+2406643]                                                                                    Ordinal0 [0x0033AF31+1945393]                                                                                    Ordinal0 [0x0022C748+837448]                                                                                     Ordinal0 [0x0025FA1B+1047067]                                                                                    Ordinal0 [0x0025D9F8+1038840]                                                                                    Ordinal0 [0x0025B6DB+1029851]                                                                                    Ordinal0 [0x0025A439+1025081]                                                                                    Ordinal0 [0x00250083+983171]                                                                                     Ordinal0 [0x0027427C+1131132]                                                                                    Ordinal0 [0x0024FA64+981604]                                                                                     Ordinal0 [0x00274494+1131668]                                                                                    Ordinal0 [0x00284682+1197698]                                                                                    Ordinal0 [0x00274096+1130646]                                                                                    Ordinal0 [0x0024E636+976438]                                                                                     Ordinal0 [0x0024F546+980294]                                                                                     GetHandleVerifier [0x00619612+2498066]                                                                           GetHandleVerifier [0x0060C920+2445600]                                                                           GetHandleVerifier [0x00444F2A+579370]                                                                            GetHandleVerifier [0x00443D36+574774]                                                                            Ordinal0 [0x00341C0B+1973259]                                                                                    Ordinal0 [0x00346688+1992328]                                                                                    Ordinal0 [0x00346775+1992565]                                                                                    Ordinal0 [0x0034F8D1+2029777]                                                                                    BaseThreadInitThunk [0x7533FA29+25]                                                                              RtlGetAppContainerNamedObjectPath [0x770A7A7E+286]                                                               RtlGetAppContainerNamedObjectPath [0x770A7A4E+238]                                                                                                                                                                        [hanif] Getting hanif connection(s)...
```